# LAB 8 - COLLABORATIVE FILTERING

This lab is comprise of 2 main sections:

 - 1.
 
 - 2.

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
np.random.seed(88)

In [14]:
movie = pd.read_csv('MovieLensFeatures.csv')
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 53 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   AgeRange                 1000209 non-null  int64  
 1   Jobacademic.educator     1000209 non-null  int64  
 2   Jobartist                1000209 non-null  int64  
 3   Jobclerical.admin        1000209 non-null  int64  
 4   Jobcollege.grad.student  1000209 non-null  int64  
 5   Jobcustomer.service      1000209 non-null  int64  
 6   Jobdoctor.health.care    1000209 non-null  int64  
 7   Jobexecutive.managerial  1000209 non-null  int64  
 8   Jobfarmer                1000209 non-null  int64  
 9   Jobhomemaker             1000209 non-null  int64  
 10  JobK.12.student          1000209 non-null  int64  
 11  Joblawyer                1000209 non-null  int64  
 12  Jobprogrammer            1000209 non-null  int64  
 13  Jobretired               1000209 non-null 

**RENAME COLUMNS**

Note some column names have dots in their names. We would need to rename these columns if wanted to use them. 
For the time being, we are only interestesd in `userID`, `movieID`, `rating`, so we will proceed with the dataset unchanged.

In [15]:
# pd.set_options()
movie[['userID', 'movieID', 'rating']].describe()

,userID,movieID,rating
count,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.731481e+03,3.581564e+00
std,1.728413e+03,1.017026e+03,1.117102e+00
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.506000e+03,9.660000e+02,3.000000e+00
50%,3.070000e+03,1.658000e+03,4.000000e+00
75%,4.476000e+03,2.566000e+03,4.000000e+00
max,6.040000e+03,3.706000e+03,5.000000e+00


In [16]:
## DEFINE OSR2?

def OSR2(model, X_test, y_test, y_train):
    
    y_pred = model.predict(X_test)
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(y_train))**2)
                 
    return (1 - SSE/SST)

### Split the Data

In [17]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(movie, train_size=95/100, random_state=88)

print(train.shape, test.shape)

# idx = int(len(movie)*95/100)
# train = movie.iloc[:idx]
# test = movie.iloc[idx:]

# print(train.shape, test.shape)

(950198, 53) (50011, 53)


Split training into real training and validation set

In [18]:
train, val = train_test_split(train, test_size=5/95, random_state=88)

print(train.shape, val.shape)

# idx = int(len(train)*5/95)
# train = train.iloc[:-idx]
# val = train.iloc[-idx:]

# print(train.shape, val.shape)

(900187, 53) (50011, 53)


In [19]:
# ## R
# > dim(test)
# [1] 50011    53
# > dim(val)
# [1] 50010    53
# > dim(train)
# [1] 900188     53

print(len(train)+len(val)+len(test), len(movie))

1000209 1000209


### Pivot Tables

First we must create the incomplete version of our user-movie ratings matrix

In [20]:
train_mat_incomplete = train.pivot('userID', 'movieID', 'rating')
train_mat_incomplete

# r = range(1, 10)
# df = (df.pivot('I1','I2', 'V')
#         .fillna(0)
#         .astype(int)
#         .reindex(index=r, columns=r, fill_value=0)
#         .rename_axis(None)
#         .rename_axis(None, axis=1))
# print (df)

movieID,1,2,3,4,5,6,7,8,9,10,...,3697,3698,3699,3700,3701,3702,3703,3704,3705,3706
userID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
train_mat_incomplete.describe()

movieID,1,2,3,4,5,6,7,8,9,10,...,3697,3698,3699,3700,3701,3702,3703,3704,3705,3706
count,1850.000000,630.000000,423.000000,148.000000,262.000000,851.000000,422.000000,61.000000,98.000000,803.000000,...,86.000000,9.000000,38.000000,85.000000,51.000000,765.000000,272.000000,48.000000,34.000000,359.000000
mean,4.146486,3.201587,3.018913,2.790541,3.007634,3.880141,3.433649,3.016393,2.653061,3.554172,...,3.081395,2.111111,1.473684,2.235294,3.509804,3.627451,4.102941,3.729167,3.882353,3.779944
std,0.859302,0.993939,1.081784,1.011857,1.024573,0.927853,0.986419,0.991494,1.056175,0.886582,...,1.229205,1.364225,1.006381,1.065107,1.172562,1.010950,1.036453,1.026035,1.094468,0.950765
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.000000,3.000000,2.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,...,2.000000,1.000000,1.000000,1.000000,3.000000,3.000000,4.000000,3.000000,3.000000,3.000000
50%,4.000000,3.000000,3.000000,3.000000,3.000000,4.000000,3.000000,3.000000,3.000000,4.000000,...,3.000000,2.000000,1.000000,2.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,5.000000,4.000000,4.000000,3.000000,4.000000,5.000000,4.000000,4.000000,3.000000,4.000000,...,4.000000,3.000000,1.000000,3.000000,4.000000,4.000000,5.000000,4.250000,5.000000,4.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


## 2. CENTERING THE MATRIX

### Biscale

In [22]:
from fancyimpute import SoftImpute, BiScaler

# X_incomplete_normalized = BiScaler().fit_transform(X_incomplete)

train_mat_centered = BiScaler().fit_transform(train_mat_incomplete)
# train_mat_centered = BiScaler(max_iters=10000, scale_rows=False, scale_columns=False).fit_transform(train_mat_incomplete)

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:
alpha = train_mat_centered.alpha
beta = train_mat_centered.beta

# take a look at mutate(), arrange(), desc(), inner_join(), functions that may be useful for HW5


# # Biscale

# mat.train <- Incomplete(train$userID, train$movieID, train$rating)
# summary(train)

# # Instruction of biscale()
# set.seed(100)
# mat.train.centered <- biScale(mat.train, maxit = 10000, row.scale = FALSE, col.scale = FALSE)
# # mat.train.centered is X_ij - alpha_i - beta_j
# alpha <- attr(mat.train.centered, "biScale:row")$center
# beta <- attr(mat.train.centered, "biScale:column")$center
# # take a look at mutate(), arrange(), desc(), inner_join(), functions that may be useful for HW5

## 3. MODEL

In [ ]:
# movie <- MovieLensFeatures

# clean up
movie$AgeRange <- as.factor(movie$AgeRange)
movie$wday <- as.factor(movie$wday)
movie$mon <- as.factor(movie$mon)
movie$year <- as.factor(movie$year)
movie$hour <- as.factor(movie$hour)

set.seed(122)
train.ids <- sample(nrow(movie), 0.95*nrow(movie))
train <- movie[train.ids,]
test <- movie[-train.ids,]

# split training into real training and validation set
val1.ids <- sample(nrow(train), (5/95)*nrow(train))
val1 <- train[val1.ids,]
train <- train[-val1.ids,]

val2.ids <- sample(nrow(train), (5/90)*nrow(train))
val2 <- train[val2.ids,]
train <- train[-val2.ids,]

# First try CF
mat.train <- Incomplete(train$userID, train$movieID, train$rating)

# compute validation set MAE for rank = 1,2,...,20
mae.vals = rep(NA, 20)
for (rnk in seq_len(20)) {
  print(str_c("Trying rank.max = ", rnk))
  mod <- softImpute(mat.train, rank.max = rnk, lambda = 0, maxit = 1000)
  preds <- impute(mod, val1$userID, val1$movieID) %>% pmin(5) %>% pmax(1)
  mae.vals[rnk] <- mean(abs(preds - val1$rating))
}

mae.val.df <- data.frame(rnk = seq_len(20), mae = mae.vals)
ggplot(mae.val.df, aes(x = rnk, y = mae)) + geom_point(size = 3) + 
  ylab("Validation MAE") + xlab("Number of Archetypal Users") + 
  theme_bw() + theme(axis.title=element_text(size=18), axis.text=element_text(size=18))


# choose k = 10
set.seed(100)
mod.final <- softImpute(mat.train, rank.max = 10, lambda = 0, maxit = 1000)
preds <- impute(mod.final, test$userID, test$movieID) %>% pmin(5) %>% pmax(1)

mean(abs(preds - test$rating))/4
sqrt(mean((preds - test$rating)^2))/4
OSR2(preds, train$rating, test$rating)


# Now try a linear regression without CF as a varible
lin.mod <- lm(rating ~ . -movieID -userID, data = train)
summary(lin.mod)

preds.lm <- predict(lin.mod, newdata = test) %>% pmin(5) %>% pmax(1)
mean(abs(preds.lm - test$rating))/4
sqrt(mean((preds.lm - test$rating)^2))/4
OSR2(preds.lm, train$rating, test$rating)


# Now try random forests (Warning: this took 2 hours to run)
set.seed(3592)
rf.mod <- ranger(rating ~ . -movieID -userID, 
                 data = train, 
                 mtry = floor((ncol(train) - 3)/3), 
                 num.trees = 500,
                 verbose = TRUE)

preds.rf <- predict(rf.mod, data = test)
preds.rf <- preds.rf$predictions
mean(abs(preds.rf - test$rating))/4
sqrt(mean((preds.rf - test$rating)^2))/4
OSR2(preds.rf, train$rating, test$rating)



# Blending

val.preds.cf <- impute(mod.final, val2$userID, val2$movieID)
val.preds.lm <- predict(lin.mod, newdata = val2)
val.preds.rf <- predict(rf.mod, data = val2)$predictions

# Build validation set data frame
val.blending_df = data.frame(rating = val2$rating, cf_preds = val.preds.cf, 
                             lm_preds = val.preds.lm, rf_preds = val.preds.rf)

# Train blended model
blend.mod = lm(rating ~ . -1, data = val.blending_df)
summary(blend.mod)

# Get predictions on test set
test.preds.cf <- impute(mod.final, test$userID, test$movieID)
test.preds.lm <- predict(lin.mod, newdata = test)
test.preds.rf <- predict(rf.mod, data = test)$predictions

test.blending_df = data.frame(rating = test$rating, cf_preds = test.preds.cf, 
                              lm_preds = test.preds.lm, rf_preds = test.preds.rf)

test.preds.blend <- predict(blend.mod, newdata = test.blending_df)

mean(abs(test.preds.blend - test$rating))/4
sqrt(mean((test.preds.blend - test$rating)^2))/4
OSR2(test.preds.blend, train$rating, test$rating)
